In [1]:
!pip install plotly

Defaulting to user installation because normal site-packages is not writeable


In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import lightgbm as lgb
import plotly as ply
import numpy as np

In [3]:
active_losses_df = pd.read_csv(
    "../data/Avtice-losses.csv", skiprows=2, names=["datetime", "MWh"], parse_dates=["datetime"], index_col="datetime")

active_losses_df.index = active_losses_df.tz_localize('Europe/Brussels', ambiguous="infer").tz_convert('UTC').index
active_losses_df.index = active_losses_df.index - pd.Timedelta(minutes=15)
active_losses_df = active_losses_df.resample('1H').sum()
active_losses_df["MWh"] = active_losses_df["MWh"]/1000


df = active_losses_df


# Predict next hour based on last 7 days

In [4]:
feature_columns = ['MWh']

In [5]:
# Create lagged features
for col in feature_columns:
    for i in range(1, 7 * 24 + 1):  # for past 7 days (in hours)
        df[f'lag_{col}_{i}'] = df[col].shift(i)
df['month'] = (df.index.month).astype(int)

/tmp/ipykernel_143381/2628811236.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'lag_{col}_{i}'] = df[col].shift(i)
/tmp/ipykernel_143381/2628811236.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'lag_{col}_{i}'] = df[col].shift(i)
/tmp/ipykernel_143381/2628811236.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame

In [6]:
df

,MWh,lag_MWh_1,lag_MWh_2,lag_MWh_3,lag_MWh_4,lag_MWh_5,lag_MWh_6,lag_MWh_7,lag_MWh_8,lag_MWh_9,...,lag_MWh_160,lag_MWh_161,lag_MWh_162,lag_MWh_163,lag_MWh_164,lag_MWh_165,lag_MWh_166,lag_MWh_167,lag_MWh_168,month
datetime,,,,,,,,,,,,,,,,,,,,,
2018-12-31 23:00:00+00:00,139.525004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
2019-01-01 00:00:00+00:00,129.716036,139.525004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2019-01-01 01:00:00+00:00,133.398074,129.716036,139.525004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2019-01-01 02:00:00+00:00,135.133852,133.398074,129.716036,139.525004,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2019-01-01 03:00:00+00:00,131.699424,135.133852,133.398074,129.716036,139.525004,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 18:00:00+00:00,171.707318,152.159763,162.995636,188.541866,176.617953,167.343565,145.792085,152.648212,150.967027,149.349078,...,119.124221,115.795738,103.814345,97.836516,97.016161,103.763064,105.380492,103.315829,99.421761,12
2021-12-31 19:00:00+00:00,159.462903,171.707318,152.159763,162.995636,188.541866,176.617953,167.343565,145.792085,152.648212,150.967027,...,125.661865,119.124221,115.795738,103.814345,97.836516,97.016161,103.763064,105.380492,103.315829,12
2021-12-31 20:00:00+00:00,155.109520,159.462903,171.707318,152.159763,162.995636,188.541866,176.617953,167.343565,145.792085,152.648212,...,136.132174,125.661865,119.124221,115.795738,103.814345,97.836516,97.016161,103.763064,105.380492,12


In [7]:
len(df)

26304

In [8]:
X = df.drop('MWh', axis=1)  # Features
y = df['MWh']  # Target variable

# Split the data into train and test sets
X_train = X[:21000]
y_train = y[:21000]
X_test = X[21001:]
y_test = y[21001:]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Define the parameters for LightGBM
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'device': 'gpu'  # Enable GPU support
}

# Create the LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
# Train the model
model = lgb.train(params, train_data)

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 42853
[LightGBM] [Info] Number of data points in the train set: 21000, number of used features: 169
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 169 dense feature groups (3.44 MB) transferred to GPU in 0.041687 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 107.649150


In [10]:
from sklearn import metrics

y_pred = model.predict(X_test)
metrics.mean_absolute_error(y_test, y_pred)

8.189550518894444

In [12]:
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = metrics.mean_absolute_error(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

RMSE: 10.768973898822182
MAE: 8.189550518894444


# Using Vectors

In [ ]:
# Create a new DataFrame with a date index
date_index = active_losses_df.index.date
new_df = pd.DataFrame(index=date_index)
new_df.index = new_df.index.unique()

In [ ]:
# Initialize an empty array for each date with 24 values
new_df['MWh'] = [np.array([np.nan] * 24) for _ in range(len(new_df))]

In [ ]:
# Populate the array with the hourly values
for date in new_df.index:
    hourly_values = active_losses_df.loc[date.strftime('%Y-%m-%d')]['MWh'].values
    new_df.at[date, 'MWh'][:len(hourly_values)] = hourly_values
    
new_df

In [ ]:
# Populate the array with the hourly values
for date in date_index:
    #hourly_values = active_losses_df.loc[date.strftime('%Y-%m-%d')]['MWh'].values
    #new_df.at[date, 'MWh'][:len(hourly_values)] = hourly_values
    
    date_values = active_losses_df.loc[date.strftime('%Y-%m-%d')]
    #print(date_values)
    #print(date_values['MWh'].values)
    new_df.loc[date] = date_values.values

# Optional: Fill NaN values with zeros
new_df['MWh'] = new_df['MWh'].apply(lambda x: np.nan_to_num(x, nan=0))

# Print the resulting DataFrame
#print(new_df) 

In [ ]:
df.reset_index()

In [ ]:
# Adding a second index that repeats from 1 to 8
df['sample_index'] = (df.reset_index().index // 8) + 1

In [ ]:
df

In [6]:
new_df = pd.read_csv("df.csv")
new_df

,datetime,solar_fore_de_MW,solar_fore_it_MW,wind_fore_de_MW,wind_fore_it_MW,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it,Month_1,...,lag_mwh_3,lag_mwh_4,lag_mwh_5,lag_mwh_6,lag_mwh_7,lag_mwh_8,lag_mwh_9,lag_mwh_10,lag_mwh_11,lag_mwh_12
0,2019-01-01 12:00:00+00:00,2623.0,4722.0,34266.1751,4414.9774,4.882300,8.647100,7.459600,7.738600,True,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,2019-01-01 13:00:00+00:00,1775.0,3595.0,35997.4125,3952.2944,4.808495,8.582632,7.264148,7.570064,True,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,2019-01-01 14:00:00+00:00,722.0,1965.0,37841.5032,3476.5546,4.524808,8.317492,6.936318,7.047034,True,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,2019-01-01 15:00:00+00:00,41.0,343.0,37485.1176,3050.5234,4.126172,7.928399,6.524037,6.294940,True,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,2019-01-01 16:00:00+00:00,0.0,0.0,38010.4885,2790.4242,3.707520,7.492072,6.075238,5.439215,True,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26286,2021-12-31 18:00:00+00:00,0.0,0.0,36997.7200,1108.4000,8.530000,9.960000,10.790000,9.590000,False,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
26287,2021-12-31 19:00:00+00:00,0.0,0.0,35666.9300,1077.9700,8.000000,9.400000,10.630000,9.110000,False,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
26288,2021-12-31 20:00:00+00:00,0.0,0.0,34383.8800,1048.2800,7.500000,8.880000,10.510000,8.670000,False,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
26289,2021-12-31 21:00:00+00:00,0.0,0.0,33075.2500,1078.7800,6.970000,8.510000,10.320000,8.140000,False,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [12]:
new_df.columns

Index(['datetime', 'solar_fore_de_MW', 'solar_fore_it_MW', 'wind_fore_de_MW',
       'wind_fore_it_MW', 'temperature_fore_ch', 'temperature_fore_fr',
       'temperature_fore_de', 'temperature_fore_it', 'Month_1', 'Month_2',
       'Month_3', 'Month_4', 'Month_5', 'Month_6', 'Month_7', 'Month_8',
       'Month_9', 'Month_10', 'Month_11', 'Month_12', 'Day_0', 'Day_1',
       'Day_2', 'Day_3', 'Day_4', 'Day_5', 'Day_6', 'Hour_0', 'Hour_1',
       'Hour_2', 'Hour_3', 'Hour_4', 'Hour_5', 'Hour_6', 'Hour_7', 'Hour_8',
       'Hour_9', 'Hour_10', 'Hour_11', 'Hour_12', 'Hour_13', 'Hour_14',
       'Hour_15', 'Hour_16', 'Hour_17', 'Hour_18', 'Hour_19', 'Hour_20',
       'Hour_21', 'Hour_22', 'Hour_23', 'target', 'std_mwh', 'lag_mwh_1',
       'lag_mwh_2', 'lag_mwh_3', 'lag_mwh_4', 'lag_mwh_5', 'lag_mwh_6',
       'lag_mwh_7', 'lag_mwh_8', 'lag_mwh_9', 'lag_mwh_10', 'lag_mwh_11',
       'lag_mwh_12'],
      dtype='object')